# WORK in progress!!!

# GGM omzetten naar MIM

Dit notebook wordt gebruikt om de bestaande versie 1 van [Gemeentelijk Gegevensmodel](https://github.com/Gemeente-Delft/Gemeentelijk-Gegevensmodel) om te zetten naar [MIM (MIM - Metamodel Informatie Modellering)](https://docs.geostandaarden.nl/mim/mim/). 

Hiervoor is het GGM ingelezen in de bijgevoegde Postgres-database en worden op basis van Datamanipulatie de juiste tags en attributen gezet voor: 

1. objecttypes
2. packages
3. enumeraties
4. waardelijsten
5. relaties (associaties en generalisaties)

En vervolgs worden de onderliggende attributen van de juiste tags en attributen voorzien.

Om te kiezen welk onderdeel van het GGM je wil omzetten kies je het GUID (root_guid onder 'Configuratie') van het bijbehorende package. Alles wat daaronder zit zet deze app om. Met name voor test en ontwikkeling is het mogelijk gemaakt sledchts delen om te zetten.    

In [1]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import json
from IPython.display import JSON as JSONDisplay
import requests
import database
from re import sub
import ast
import re

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Sequence
from sqlalchemy.dialects.postgresql import insert
import uuid

def generateEAUUID():
    return '{' +  str(uuid.uuid4()).upper() + '}'

## Configuratie

Met de waarden hieronder configureer je de omzetteing. Als voorbeeld is het GUID van Onderwijs actief. Bij de algehele omzetting zal de root_guid gelijk zijn aan de guid_ggm. Guid_ggm is nodig om de juiste tags en attribute te zetten in het hoofdmodel. 

In [4]:
### Configuratie
db_uri = os.getenv("DATABASE_GGM_URL")
excel_concepten = "sqlite://./root/gemeentelijk gegevensmodel EA16.qea" #'./data/Concepten GGM.xlsx'

root_guid = '{88E4851C-DACE-4cec-9B14-A36FA6D3622E}' #Onderwijs in package ontwikkeling
root_guid = '{DFF92641-96B6-465d-A239-9BE0B5208281}' #Monumenten
root_guid = '{981576A5-D37A-4f65-8CDB-FF4D4BB06D69}' # Root GGM

guid_ggm = '{981576A5-D37A-4f65-8CDB-FF4D4BB06D69}'
guid_voorbeelden = ['{8F96CFC6-DBD3-453b-8866-CD34F76A3CCE}', '{48334EBF-FF19-4b0b-9B47-D0BE403DC80D}']

## Inlezen Model

Hieronder wordt het model (vanaf 'root_guid' en alles daaronder) ingelezen in het dataframe df voor verdere verwerking. Achtereenvolgens worden ingelezen:

1. alle objecten (packages, classes, enumeraties en waardelijsten)
2. alle attributen bij deze objecten
3. alle datatypes waaraan gerelateerd gaat worden

In [5]:
# alle objecten (packages, classes, enumeraties en waardelijsten)

df_obj = database.get_df_objectsHierar(db_uri, root_guid=root_guid)
df_obj['stereotype_object'] = df_obj['stereotype']
df_obj.head(5)

AttributeError: 'NoneType' object has no attribute '_instantiate_plugins'

In [4]:
# Lees de GEMMA-concepten, zodat verbinding gelegd kan worden met de GEMMA-bedrijfsobjecten 

df_concepten = pd.read_excel(excel_concepten)
df_concepten.rename(columns={'Object ID': 'GEMMA object_id', 
                             'Concept': 'name'}, inplace=True)
df_concepten.head(5)

,GEMMA link,GEMMA object_id,name,Documentation,Bron,Business Object,Objecttype,Model
0,https://gemmaonline.staging.wikixl.nl/index.ph...,a95380d8-9c99-5986-7e7f-50e69bc02eda,AANDUIDING VERBLIJFSRECHT,Aanduiding in verband met het verblijfsrecht v...,https://www.gemmaonline.nl/index.php/Informati...,Ja,Ja,Model Relatielijsten (naam?)
1,https://gemmaonline.staging.wikixl.nl/index.ph...,0c5a13bd-581f-5163-6cff-ed742422679f,AARD AANTEKENING,Een opsomming van de diverse aarden van de aan...,NaN,Nee,Ja,Model Relatielijsten (naam?)
2,https://gemmaonline.staging.wikixl.nl/index.ph...,9a98c324-209a-a3db-8a0f-fe20eb2964a4,AARD FILIATIE,Een opsomming van redenen waarom kadastrale on...,NaN,Nee,Ja,Model Relatielijsten (naam?)
3,https://gemmaonline.staging.wikixl.nl/index.ph...,69b6509c-4f16-1d54-2d92-ad9070bf4942,AARD ZAKELIJK RECHT,Een opsomming van de diverse aarden van het za...,NaN,Nee,Ja,Model Relatielijsten (naam?)
4,https://gemmaonline.staging.wikixl.nl/index.ph...,1f40efea-23c4-77c1-599c-ae1c38e42fdf,ACADEMISCHE TITEL,Een formeel via het Staatsblad gepubliceerde a...,NaN,Nee,Ja,Model Relatielijsten (naam?)


In [5]:
df_obj = df_obj.merge(df_concepten, on='name', how='left')
df_obj.head(5)

,object_id,object_type,stereotype,name,alias,author,version,objectnote,ea_guid,modifieddate,tree,stereotype_object,GEMMA link,GEMMA object_id,Documentation,Bron,Business Object,Objecttype,Model
0,5641,Class,Objecttype,Archeologiebesluit,None,Arjen Brienen,1.0,None,{AD181233-EEE8-4165-87D0-52441D8A1C02},2022-12-06 14:17:33,1145-1144-1136-165,Objecttype,https://gemmaonline.staging.wikixl.nl/index.ph...,54189b69-9591-3fd9-a2b5-a9d3f60020c8,Een professioneel oordeel dat gebaseerd is op ...,NaN,Ja,Ja,"Model Archeologie,Archeologie Domeinmodel"
1,5644,Class,Objecttype,boring,None,Arjen Brienen,1.0,None,{88260958-62BB-4fba-B4E6-4EF3D155A2BE},2022-12-06 14:17:35,1145-1144-1136-165,Objecttype,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5648,Class,Objecttype,Magazijnlocatie,None,Arjen Brienen,1.0,None,{1ECDC600-15E9-402c-A69F-0D6180602FA4},2022-12-06 14:17:36,1145-1144-1136-165,Objecttype,https://gemmaonline.staging.wikixl.nl/index.ph...,eb8d6fb1-7b02-2c62-3e38-181a068a1334,Locatie van een magazijn,NaN,Nee,Ja,NaN
3,5649,Class,Objecttype,Magazijnplaatsing,None,Arjen Brienen,1.0,None,{8EA329AB-2139-4ceb-B5A5-FD04CC1D8D97},2022-12-06 14:17:36,1145-1144-1136-165,Objecttype,https://gemmaonline.staging.wikixl.nl/index.ph...,18098002-44f7-8610-019b-a361608f7983,Het ergens neerzetten van een object in een ma...,NaN,Nee,Ja,NaN
4,5653,Class,Objecttype,Stelling,None,Arjen Brienen,1.0,None,{AC825EAB-BF3C-4ecd-9D9E-7D5D5D911958},2022-12-06 14:17:37,1145-1144-1136-165,Objecttype,https://gemmaonline.staging.wikixl.nl/index.ph...,4a78754d-6ea6-76dd-8a82-f9d9f32896e6,Een systeem om goederen op te slaan die worden...,https://thesaurus.cultureelerfgoed.nl/concept/...,Ja,Ja,"Model Archeologie,Model Archief,Archeologie Do..."


In [6]:
# inlezen alle attributen bij deze objecten

df_attr = database.get_df(db_uri, "select * from t_attribute")
df_attr = df_attr.merge(df_obj[['object_id', 'object_type', 'stereotype_object']], on='object_id', how='inner')
#df_attr[df_attr.object_type == 'Referentielijst'].head(5)
df_attr.head(5)

,object_id,name,scope,stereotype,containment,isstatic,iscollection,isordered,allowduplicates,lowerbound,...,scale,const,style,classifier,Default,type,ea_guid,styleex,object_type,stereotype_object
0,5680,soort,Public,None,Not Specified,0,0,0,0,1,...,NaN,0,None,0,Column(String(80)),AN80,{DE30B2B1-082F-43f4-855B-B6DB846D6D7B},volatile=0;union=0;,Class,None
1,5680,aantal,Public,None,Not Specified,0,0,0,0,1,...,NaN,0,None,0,Column(Integer),int,{E70F3BEF-79ED-4fbc-87E0-AF4A05C98E37},volatile=0;union=0;,Class,None
2,5680,primary_key,Public,None,Not Specified,0,0,0,0,1,...,NaN,0,None,0,"Column(Integer, primary_key=True, autoincremen...",var,{2D3BFADB-476E-4308-92A0-3AD4B3A0ABC2},volatile=0;union=0;,Class,None
3,5680,monsterID,Public,None,Not Specified,0,0,0,0,1,...,NaN,0,None,0,"Column(ForeignKey('Def_Monster.primary_key', d...",var,{1F292C22-8E90-4ebe-9D04-0268CDAE622E},volatile=0;union=0;,Class,None
4,5680,monster,Public,None,Not Specified,0,0,0,0,1,...,NaN,0,None,0,"relationship('Monster', backref=""schelpmateria...",var,{ED5208A9-B83E-41e4-85D4-1792203F341C},volatile=0;union=0;,Class,None


In [6]:
# inlezen alle relaties bij deze objecten
# alle relaties (associaties en generatiesaties) die bij start of einde een object uit de subset hebben worden meegenomen

df_con = database.get_df(db_uri, "select * from t_connector where connector_type IN (\'Association\', \'Generalization\')")
df_con_start = df_con.merge(df_obj[['object_id']], right_on='object_id', left_on='start_object_id', how='inner')
df_con_end = df_con.merge(df_obj[['object_id']], right_on='object_id', left_on='end_object_id', how='inner')

df_con = pd.concat([df_con_start, df_con_end], ignore_index=True)
df_con.drop_duplicates('connector_id', inplace=True)
df_con.head(5)

AttributeError: 'NoneType' object has no attribute '_instantiate_plugins'

In [8]:
# inlezen alle datatypes om aan te relateren

df_datatypes = database.get_df(db_uri, "select * from t_object where object_type = 'DataType'")
df_datatypes[df_datatypes.name.str.contains('edrag')].head(5)

,object_id,object_type,diagram_id,name,alias,author,version,note,package_id,stereotype,...,isroot,isleaf,isspec,isactive,stateflags,packageflags,multiplicity,styleex,actionflags,eventflags
0,5415,DataType,0,TypeBedrag,None,Arjen Brienen,1.0,Een bedrag is een getal met twee decimalen in ...,450,Primitief datatype,...,0,0,0,0,None,None,None,None,None,None


## Uitvoeren omzetting

Hieronder vindt de omzetting plaats. Achtereenvolgens worden de volgende stappen gezet:

1. omzetten algemene packages 
2. omzetten alle objecten onder root_guid
3. omzetten alle relaties verbonden aan deze objecten
4. omzetten alle attributen van deze objecten

Bij iedere omzetting worden de volgende waarden gezet:
    
1. attributen van het object conform MIM, waaronder altijd het stereotype
2. een xref-record dat de relatie legt tussen het MIM-concept van het stereotype en het zojuist gezette stereotype. in het xref-record wordt steeds de waarde van het mim_text_... opgenomen.
3. de tags/properties die conmform MIM aan een object toegekend moeten worden. Deze staan steeds opgenomen in de 'lst_prop'-lijsten. 

In [9]:
# Omzetten algemene packages
# Generic function to set object_types
def changePackage(session, ea_guid, object_type, mim_text, lst_prop):
    t_obj = session.query(Object).filter_by(ea_guid=ea_guid).first()
    #print(f'Changing {t_obj.name}')
    setattr(t_obj, 'stereotype', 'Basismodel')
    if ea_guid == guid_ggm:
        setattr(t_obj, 'objectnote', note_ggm)

    t_xref = session.query(XRef).filter_by(client=ea_guid, description=mim_text_basismodel).first()
    if not t_xref:
        session.add(XRef(xrefid=generateEAUUID(), name='Stereotypes', type='element property', visibility='Public', partition=0, client=ea_guid, description=mim_text_basismodel))

    for prop in lst_props:
        t_objprop = session.query(Objectproperty).filter_by(object_id=t_obj.object_id, property=prop['property']).first()
        if not t_objprop:
            new_id = engine.execute(Sequence('propertyid_seq'))
            session.add(Objectproperty(propertyid=new_id, object_id=t_obj.object_id, property=prop['property'], value=prop['value'], notes=prop['notes'], ea_guid=generateEAUUID()))


note_ggm = '''Het Gemeentelijk Gegevensmodel (GGM) is een logisch gegevensmodel met daarin vertegenwoordigd alle beleidsterreinen van de gemeente. Het GGM is ontwikkeld in opdracht van de Gemeente Delft ter ondersteuning van de visie op het gebied van informatiegestuurd werken. Onder andere wordt het GGM gebruikt als centraal datamodel in het datawarehouse. Hiertoe is een generator beschikbaar om het GGM te vertalen naar fysieke databasetabellen.'''

mim_text_basismodel = '@STEREO;Name=Basismodel;FQName=VNGR SIM+Grouping NL::Basismodel;@ENDSTEREO;'
lst_props = [{'property': 'Afkorting',
  'value': None,
  'notes': 'Description: Afkorting van dit model. Deze afkorting wordt o.a. gebruikt waar informatie over het model wordt gepubliceerd.\r\n'},
 {'property': 'Alternatieve naam',
  'value': None,
  'notes': 'Description: De naam in natuurlijke of formele taal; afhankelijk van gekozen aanpak. Een alternatieve naam.\r\n'},
 {'property': 'derived', 'value': 'yes', 'notes': None},
 {'property': 'Domein', 'value': None, 'notes': None},
 {'property': 'Imvertor',
  'value': 'model',
  'notes': 'Values: model\r\nDefault: model\r\n'},
 {'property': 'Informatiedomein',
  'value': 'Gemeentelijk Informatiehuishouding',
  'notes': None},
 {'property': 'Informatiemodel type', 'value': 'Logisch', 'notes': None},
 {'property': 'Is afgeleid',
  'value': 'Nee',
  'notes': 'Values: Nee,Ja,Zie package\r\nDefault: Zie package\r\nDescription: Deze constructie is al dan niet afgeleid van een "supplier model". Wanneer je niks opgeeft wordt afleiding vastgesteld op basis van het package waarin het voorkomt.\r\n'},
 {'property': 'MIM extensie', 'value': None, 'notes': None},
 {'property': 'MIM taal', 'value': None, 'notes': None},
 {'property': 'MIM versie', 'value': '1.1', 'notes': None},
 {'property': 'Niveau',
  'value': 'specifiek',
  'notes': 'Values: generiek,specifiek\r\nDefault: specifiek\r\nDescription: Het niveau is een waarde uit een beperkte set: "algemeen" betekent dat het model een fundament is van een ander model en moet worden opgenomen. Het modelleert generieke constructies die deel uitmaken van de echte wereld. Dit omvat meestal abstracte object typen. Het kan andere constructen bevatten. "specifiek" betekent dat het model een bepaald domein beschrijft of een samenhangend deel ervan .\r\n'},
 {'property': 'Relatiemodelleringstype', 'value': 'Relatie', 'notes': None},
 {'property': 'release',
  'value': '1.1',
  'notes': 'Description: Dit bevat de releasedatum in het format yyyymmdd . De releasedatum wordt mede gebruikt om een model, koppelvlak of bericht uniek te identificeren in Imvertor.\r\n'},
 {'property': 'Versie ID',
  'value': None,
  'notes': 'Description: De identificatie van de versie/revisie van dit model of model-element in het gehanteerde versiebeheersysteem.\r\n'},
 {'property': 'Web locatie', 'value': 'https://github.com/gemeente-delft/Gemeentelijk-Gegevensmodel/', 'notes': None}]

# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
Object = Base.classes.t_object
XRef = Base.classes.t_xref
Objectproperty = Base.classes.t_objectproperties

# Do transformation
session = Session(engine)

for guid in ([guid_ggm]): # + guid_voorbeelden
    changePackage(session, guid, 'Basismodel', mim_text_basismodel, lst_props)

session.commit()
engine.dispose()

In [10]:
# Omzetten alle objecten onder root_guid

# Generic function to set object_types
def changeObject(session, ea_guid, object_type, mim_text, lst_prop, row):
    t_obj = session.query(Object).filter_by(ea_guid=ea_guid).first()
    if t_obj.author == 'crossover':
        setattr(t_obj, 'author', 'Arjen Brienen')
    setattr(t_obj, 'stereotype', object_type)

    t_xref = session.query(XRef).filter_by(client=ea_guid, description=mim_text).first()
    if not t_xref:
        session.add(XRef(xrefid=generateEAUUID(), name='Stereotypes', type='element property', visibility='Public', partition=0, client=ea_guid, description=mim_text))
        
    for prop in lst_prop:
        t_objprop = session.query(Objectproperty).filter_by(object_id=t_obj.object_id, property=prop['property']).first()
        if not t_objprop:
            new_id = engine.execute(Sequence('propertyid_seq'))
            if prop['property'] == 'GEMMA Businessobject':
                propvalue = row['Business Object']
                #print(f'setting Business Object for {t_obj.name}')
            elif prop['property'] == 'GEMMA uri':
                propvalue = row['GEMMA link']
            elif prop['property'] == 'Toelichting':
                propvalue = str(row['Documentation'])[0:254]
            else:
                propvalue = prop['Default_Value']
            session.add(Objectproperty(propertyid=new_id, object_id=t_obj.object_id, property=prop['property'], value=propvalue, notes=prop['notes'], ea_guid=generateEAUUID()))
        else:
            if t_objprop.property == 'Toelichting' and (t_objprop.value == '' or not t_objprop.value or t_objprop.value == prop['Default_Value']):
                t_objprop.value = str(row['Documentation'])[0:254]
            elif prop['property'] == 'GEMMA Businessobject':
                t_objprop.value = row['Business Object']
            elif prop['property'] == 'GEMMA uri':
                t_objprop.value = row['GEMMA link']

    #print(f'Changing Object {t_obj.name} new type: {object_type}')
      
# set config
mim_text_objecttype = '@STEREO;Name=Objecttype;FQName=MIM::Objecttype;@ENDSTEREO;'
mim_text_enumeratie = '@STEREO;Name=Enumeratie;FQName=MIM::Enumeratie;@ENDSTEREO;'
mim_text_reflijst = '@STEREO;Name=Referentie element;FQName=MIM::Referentie element;@ENDSTEREO;'
mim_text_package = '@STEREO;Name=Domein;FQName=MIM::Domein;@ENDSTEREO;'
lst_prop_obj = [{'property':'Begrip', 'Default_Value': '', 'notes': 'Description: Verwijzing naar een begrip. De verwijzing heeft de vorm van een term of een URI.'},
            {'property':'Herkomst', 'Default_Value': 'Gemeente Delft', 'notes': 'Description: De registratie of het informatiemodel waaraan het informatie-element ontleend is dan wel de eigen organisatie indien het door de eigen organisatie toegevoegd is.'},
            {'property':'Herkomst definitie', 'Default_Value': 'Gemeente Delft', 'notes': 'Description: De registratie of het informatiemodel waaruit de definitie is overgenomen dan wel een aanduiding die aangeeft uit welke bronnen de definitie is samengesteld.'},
            {'property':'Datum opname', 'Default_Value': '1 januari 2020', 'notes': 'Description: De datum waarop het informatie-element is opgenomen in het informatiemodel.'},
            {'property':'Populatie', 'Default_Value': '<memo>', 'notes': 'Voor objecttypen die deel uitmaken van een (basis)registratie betreft dit de beschrijving van de exemplaren van het gedefinieerde objecttype die in de desbetreffende (basis)-registratie voorhanden zijn.'},
            {'property':'Kwaliteit', 'Default_Value': '', 'notes': 'Description: Voor objecttypen die deel uitmaken van een registratie betreft dit de waarborgen voor de juistheid van de in de registratie opgenomen objecten van het desbetreffende type.'},
            {'property':'Toelichting', 'Default_Value': '<memo>', 'notes': 'Een inhoudelijke toelichting op de toepassing van het informatie-element.'},
            {'property':'GEMMA Businessobject', 'Default_Value': 'Nee', 'notes': 'Is opgenomen als businessobject in de GEMMA-architectuur.'},
            {'property':'GEMMA uri', 'Default_Value': '', 'notes': 'Uri naar het GEMMA-businessobject.'}]
lst_prop_enum = [{'property':'Begrip', 'Default_Value': '', 'notes': 'Description: Verwijzing naar een begrip. De verwijzing heeft de vorm van een term of een URI.'}]
lst_prop_reflijst = [{'property':'Begrip', 'Default_Value': '', 'notes': 'Description: Verwijzing naar een begrip. De verwijzing heeft de vorm van een term of een URI.'},
            {'property':'Herkomst', 'Default_Value': 'Gemeente Delft', 'notes': 'Description: De registratie of het informatiemodel waaraan het informatie-element ontleend is dan wel de eigen organisatie indien het door de eigen organisatie toegevoegd is.'},
            {'property':'Datum opname', 'Default_Value': '1 januari 2020', 'notes': 'Description: De datum waarop het informatie-element is opgenomen in het informatiemodel.'},
            {'property':'Locatie', 'Default_Value': '', 'notes': 'Description: Voor objecttypen die deel uitmaken van een registratie betreft dit de waarborgen voor de juistheid van de in de registratie opgenomen objecten van het desbetreffende type.'},
            {'property':'Toelichting', 'Default_Value': '<memo>', 'notes': 'Een inhoudelijke toelichting op de toepassing van het informatie-element.'}]




# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
Object = Base.classes.t_object
XRef = Base.classes.t_xref
Objectproperty = Base.classes.t_objectproperties


# do transformation
session = Session(engine)

for index, row in df_obj.iterrows():
    if row.stereotype_object == 'Referentielijst':
        changeObject(session, row.ea_guid, 'Referentielijst', mim_text_reflijst, lst_prop_reflijst, row)
    elif row.object_type == 'Class':
        changeObject(session, row.ea_guid, 'Objecttype', mim_text_objecttype, lst_prop_obj, row)
    elif row.object_type == 'Enumeration':
        changeObject(session, row.ea_guid, 'Enumeratie', mim_text_enumeratie, lst_prop_enum, row)
    elif row.object_type == 'Package' and 'Model' in str(row['name']):
        changeObject(session, row.ea_guid, 'Domein', mim_text_package, lst_prop_enum, row)
        
session.commit()
engine.dispose()

In [11]:
# Omzetten alle attributen van deze objecten

# Dict-list die in source de regex bevat om te matchen op de originele datatypes, en in target het uiteindelijke datatype heeft staan. 
# Als er length staat wordt deze als defaultwaarde meegenomen. Als de regex een capturing group heeft is deze bedoeld om de length af te leiden, en staat deze in 'regex_group'   
lst_attr_mapping = [{'source': r'(?:int|long|short)', 'target': 'Integer'},
                     {'source': r'N(\(?[0-9]*\)?)', 'target': 'Integer', 'regex_group': 1},
                     {'source': r'te(?:x|ks)t', 'target': 'CharacterString'},
                     {'source': r'char', 'target': 'CharacterString', 'length': 1},
                     {'source': r'AN(\(?[0-9]*\)?)', 'target': 'CharacterString', 'regex_group': 1},
                     {'source': r'N[0-9]+,[0-9]+', 'target': 'Decimal'},
                     {'source': r'(?:time|tijd)', 'target': 'DateTime'},
                     {'source': r'boolean', 'target': 'Boolean'},
                     {'source': r'(?:double|float)', 'target': 'Decimal'},
                     {'source': r'bedrag', 'target': 'TypeBedrag'},
                     {'source': r'dat(?:e|um)', 'target': 'Date'},
                     {'source': r'guid', 'target': 'CharacterString', 'length': 40},
                     {'source': r'(?:gml|punt|point)', 'target': 'Point'},
                    ]
# Initializing attr mapping, add datatypes
for attr_mapping in lst_attr_mapping:
    print(attr_mapping)
    row = df_datatypes[df_datatypes.name == attr_mapping['target']].iloc[0]
    attr_mapping['target_id'] = int(row['object_id'])


# Generieke functie om attributen te wijzigen
def changeAttribute(session, ea_guid, attribuut_type, mim_text, lst_prop):
    target = None
    targetid = None
    lengte = None
    
    t_attr = session.query(Attribute).filter_by(ea_guid=row.ea_guid).first()
    
    #First set attribute stereotype
    setattr(t_attr, 'stereotype', attribuut_type)
    t_xref = session.query(XRef).filter_by(client=ea_guid, description=mim_text).first()
    if not t_xref:
        session.add(XRef(xrefid=generateEAUUID(), name='Stereotypes', type='attribute property', visibility='Public', partition=0, client=ea_guid, description=mim_text))
    
    if row['type'] and row['type'] != '':
        for mapping in lst_attr_mapping:
            match = re.match(mapping['source'], row['type'], re.IGNORECASE)
            if match:
                if 'regex_group' in mapping.keys():
                    lengte = match.group(1)
                target = mapping['target']
                targetid = mapping['target_id']
                break
                
        setattr(t_attr, 'type', target)     
        setattr(t_attr, 'classifier', targetid)   
    
    #print(f"Changing attribute {row['name']}, Type: {row['type']} target: {target} targetid: {targetid} {('lengte: ' + lengte) if lengte else ''}")    
    for prop in lst_attr_prop:
        t_objprop = session.query(AttributeTag).filter_by(elementid=t_attr.id, property=prop['property']).first()
        if not t_objprop:
            new_id = engine.execute(Sequence('propertyid_seq')) #t_attributetag
            session.add(AttributeTag(propertyid=new_id, elementid=t_attr.id, property=prop['property'], value=prop['value'] if prop['property'] != 'Lengte' else lengte, notes=prop['notes'], ea_guid=generateEAUUID()))


# set config
mim_text_attribuutsoort = '@STEREO;Name=Attribuutsoort;FQName=MIM::Attribuutsoort;@ENDSTEREO;'
mim_text_referentieelement = '@STEREO;Name=Referentie element;FQName=MIM::Referentie element;@ENDSTEREO;'
lst_attr_prop = [{'property': 'Begrip',
  'value': None,
  'notes': 'Description: Verwijzing naar een begrip. De verwijzing heeft de vorm van een term of een URI.\r\n'},
 {'property': 'Herkomst',
  'value': None,
  'notes': 'Description: De registratie of het informatiemodel waaraan het informatie-element ontleend is dan wel de eigen organisatie indien het door de eigen organisatie toegevoegd is.\r\n'},
 {'property': 'Herkomst definitie',
  'value': None,
  'notes': 'Description: De registratie of het informatiemodel waaruit de definitie is overgenomen dan wel een aanduiding die aangeeft uit welke bronnen de definitie is samengesteld.\r\n'},
 {'property': 'Datum opname',
  'value': None,
  'notes': 'Description: De datum waarop het informatie-element is opgenomen in het informatiemodel.\r\n'},
 {'property': 'Lengte',
  'value': None,
  'notes': 'Description: De aanduiding van de lengte van een gegeven.\r\n'},
 {'property': 'Patroon',
  'value': '<memo>',
  'notes': 'De verzameling van waarden die gegevens van dit attribuutsoort kunnen hebben, dat wil zeggen het waardenbereik, uitgedrukt in een specifieke structuur.'},
 {'property': 'Formeel patroon',
  'value': None,
  'notes': 'Description: Zoals patroon, formeel vastgelegd (met een reguliere expressie), uitgedrukt in een formele taal die door de computer wordt herkend.\r\n'},
 {'property': 'Indicatie materiële historie',
  'value': None,
  'notes': 'Values: Ja,Nee,Zie groep\r\nDescription: Indicatie of de materiële historie van dit informatie-element te bevragen is.\r\n'},
 {'property': 'Indicatie formele historie',
  'value': None,
  'notes': 'Values: Ja,Nee,Zie groep\r\nDescription: Indicatie of de formele historie van dit informatie-element te bevragen is.\r\n'},
 {'property': 'Authentiek',
  'value': 'Authentiek',
  'notes': 'Values: Authentiek,Basisgegeven,Wettelijk gegeven,Landelijk kerngegeven,Overig\r\nDescription: Aanduiding of het een authentiek gegeven betreft.\r\n'},
 {'property': 'Indicatie classificerend',
  'value': 'Nee',
  'notes': 'Default: Nee\r\nDescription: Indicatie dat een attribuutsoort het objecttype waar het bijhoort classificeert in (sub)typen.\r\n'},
 {'property': 'Mogelijk geen waarde',
  'value': 'Ja',
  'notes': 'Values: Ja,Nee\r\nDefault: Nee\r\nDescription: Aanduiding dat dit informatie-element geen waarde kan bevatten.\r\n'},
 {'property': 'Toelichting',
  'value': '<memo>',
  'notes': 'Een inhoudelijke toelichting op de toepassing van het informatie-element.'}]

lst_prop_reflijst = [{'property': 'Begrip',
  'value': None,
  'notes': 'Description: Verwijzing naar een begrip. De verwijzing heeft de vorm van een term of een URI.\r\n'},
 {'property': 'Datum opname',
  'value': None,
  'notes': 'Description: De datum waarop het informatie-element is opgenomen in het informatiemodel.\r\n'},
 {'property': 'Lengte',
  'value': None,
  'notes': 'Description: De aanduiding van de lengte van een gegeven.\r\n'},
 {'property': 'Patroon',
  'value': '<memo>',
  'notes': 'De verzameling van waarden die gegevens van dit attribuutsoort kunnen hebben, dat wil zeggen het waardenbereik, uitgedrukt in een specifieke structuur.'},
 {'property': 'Formeel patroon',
  'value': None,
  'notes': 'Description: Zoals patroon, formeel vastgelegd (met een reguliere expressie), uitgedrukt in een formele taal die door de computer wordt herkend.\r\n'},
 {'property': 'Toelichting',
  'value': '<memo>',
  'notes': 'Een inhoudelijke toelichting op de toepassing van het informatie-element.'}]

    


# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
# mapped classes are now created with names by default
# matching that of the table name.
Attribute = Base.classes.t_attribute
Object = Base.classes.t_object
XRef = Base.classes.t_xref
AttributeTag = Base.classes.t_attributetag


# do transformation
session = Session(engine)

for index, row in df_attr.iterrows():
    if row.stereotype_object == 'Referentielijst':
        changeAttribute(session, row.ea_guid, 'Referentie element', mim_text_reflijst, lst_prop_reflijst)
    elif row.object_type == 'Class':
        changeAttribute(session, row.ea_guid, 'Attribuutsoort', mim_text_attribuutsoort, lst_attr_prop)
    #elif row.object_type == 'Enumeration':
    #    changeObject(session, row.ea_guid, 'Enumeratie', mim_text_enumeratie, lst_prop_enum)

session.commit()
engine.dispose()

{'source': '(?:int|long|short)', 'target': 'Integer'}
{'source': 'N(\\(?[0-9]*\\)?)', 'target': 'Integer', 'regex_group': 1}
{'source': 'te(?:x|ks)t', 'target': 'CharacterString'}
{'source': 'char', 'target': 'CharacterString', 'length': 1}
{'source': 'AN(\\(?[0-9]*\\)?)', 'target': 'CharacterString', 'regex_group': 1}
{'source': 'N[0-9]+,[0-9]+', 'target': 'Decimal'}
{'source': '(?:time|tijd)', 'target': 'DateTime'}
{'source': 'boolean', 'target': 'Boolean'}
{'source': '(?:double|float)', 'target': 'Decimal'}
{'source': 'bedrag', 'target': 'TypeBedrag'}
{'source': 'dat(?:e|um)', 'target': 'Date'}
{'source': 'guid', 'target': 'CharacterString', 'length': 40}
{'source': '(?:gml|punt|point)', 'target': 'Point'}


In [12]:
# Omzetten alle relaties

# Generic function to set object_types
def changeConnector(session, ea_guid, relatie_type, mim_text, lst_prop):
    t_rel = session.query(Connector).filter_by(ea_guid=ea_guid).first()
    setattr(t_rel, 'stereotype', relatie_type)

    t_xref = session.query(XRef).filter_by(client=ea_guid, description=mim_text).first()
    if not t_xref:
        session.add(XRef(xrefid=generateEAUUID(), name='Stereotypes', type='element property', visibility='Public', partition=0, client=ea_guid, description=mim_text))
        
    for prop in lst_prop:
        t_relprop = session.query(ConnectorTag).filter_by(elementid=t_rel.connector_id, property=prop['property']).first()
        if not t_relprop:
            new_id = engine.execute(Sequence('propertyid_seq'))
            session.add(ConnectorTag(propertyid=new_id, elementid=t_rel.connector_id, property=prop['property'], value=prop['value'], notes=prop['notes'], ea_guid=generateEAUUID()))

    #print(f'Changing Connector {t_rel.name} type: {relatie_type}')
      
# set config
mim_text_associatie = '@STEREO;Name=Relatiesoort;FQName=MIM::Relatiesoort;@ENDSTEREO;'
mim_text_generalisatie = '@STEREO;Name=Generalisatie;FQName=MIM::Generalisatie;@ENDSTEREO;'
lst_prop_rel = [{'property': 'Herkomst definitie',
  'value': '',
  'notes': ' De registratie of het informatiemodel waaruit de definitie is overgenomen dan wel een aanduiding die aangeeft uit welke bronnen de definitie is samengesteld.\r\n'},
 {'property': 'Mogelijk geen waarde',
  'value': 'Ja',
  'notes': 'Values: Ja,Nee\r\nDefault: Nee\r\nDescription: Aanduiding dat dit informatie-element geen waarde kan bevatten.\r\n'},
 {'property': 'Toelichting',
  'value': '<memo>',
  'notes': 'Een inhoudelijke toelichting op de toepassing van het informatie-element.'},
 {'property': 'Authentiek',
  'value': None,
  'notes': 'Values: Authentiek,Basisgegeven,Wettelijk gegeven,Landelijk kerngegeven,Overig\r\nDescription: Aanduiding of het een authentiek gegeven betreft.\r\n'},
 {'property': 'Indicatie formele historie',
  'value': None,
  'notes': 'Values: Ja,Nee,Zie groep\r\nDescription: Indicatie of de formele historie van dit informatie-element te bevragen is.\r\n'},
 {'property': 'Indicatie materiële historie',
  'value': None,
  'notes': 'Values: Ja,Nee,Zie groep\r\nDescription: Indicatie of de materiële historie van dit informatie-element te bevragen is.\r\n'},
 {'property': 'Datum opname',
  'value': None,
  'notes': 'Description: De datum waarop het informatie-element is opgenomen in het informatiemodel.\r\n'},
 {'property': 'Herkomst',
  'value': None,
  'notes': 'Description: De registratie of het informatiemodel waaraan het informatie-element ontleend is dan wel de eigen organisatie indien het door de eigen organisatie toegevoegd is.\r\n'},
 {'property': 'Begrip',
  'value': None,
  'notes': 'Description: Verwijzing naar een begrip. De verwijzing heeft de vorm van een term of een URI.\r\n'}]




# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine)
Connector = Base.classes.t_connector
XRef = Base.classes.t_xref
ConnectorTag = Base.classes.t_connectortag


# do transformation
session = Session(engine)

for index, row in df_con.iterrows():
    if row.connector_type == 'Association':
        changeConnector(session, row.ea_guid, 'Relatiesoort', mim_text_associatie, lst_prop_rel)
    elif row.connector_type == 'Generalization':
        changeConnector(session, row.ea_guid, 'Generalisatie', mim_text_generalisatie, lst_prop_rel)
        
session.commit()
engine.dispose()

## Utils

Diverse scripts voor analyse

In [13]:
df_attrtag = database.get_df(db_uri, 'select * from t_objectproperties where object_id = 5352')
#df_attrtag[df_attrtag.elementid == 8027][['property', 'value', 'notes']].to_dict(orient='records')
df_attrtag[['property', 'value', 'notes']].to_dict(orient='records')
#df_attrtag

#df = database.get_df(db_uri, "select * from t_object where ea_guid = '{85A44C13-58A2-4367-9A13-821BDB12E3B5}'")
#df

[]

In [14]:
df_xref = database.get_df(db_uri, 'select * from t_xref')
#df_xref = df_xref[df_xref.description.str.contains('ttribuut')]
#df_xref.iloc[0]['description']
#df_xref[df_xref.client =='{8CCDDD77-C7F3-4e4b-B429-8DDC576E42A3}'].iloc[0]['description']
df_xref.head(5)

df_xref[df_xref.description.str.contains('Gene')]['description'].unique()

array(['@STEREO;Name=Generalisatie;FQName=MIG::Generalisatie;@ENDSTEREO;',
       '@STEREO;Name=Generalisatie;GUID={8D42E571-78DA-4353-8821-53E19183B183};@ENDSTEREO;',
       '@STEREO;Name=Generalisatie;GUID={93E07441-5718-46a3-AE76-48C6C08D5BAC};@ENDSTEREO;',
       '@STEREO;Name=Generalisatie;GUID={C4A5E6D7-577A-4bb3-8B61-E8256132794B};@ENDSTEREO;',
       '@STEREO;Name=Generalisatie;FQName=MIM::Generalisatie;@ENDSTEREO;'],
      dtype=object)

In [15]:
df_prop = database.get_df(db_uri, 'select * from t_objectproperties')
df_prop[df_prop.property.str.contains('GEMMA')]

,propertyid,object_id,property,value,notes,ea_guid
56,187150,5658,GEMMA Businessobject,NaN,Is opgenomen als businessobject in de GEMMA-ar...,{FC949D71-901A-47C4-8F44-D44F20B58F4F}
230,187285,5672,GEMMA Businessobject,Ja,Is opgenomen als businessobject in de GEMMA-ar...,{3C071EF4-3982-4790-9583-674FE1CCE662}
231,187286,5672,GEMMA uri,https://gemmaonline.staging.wikixl.nl/index.ph...,Uri naar het GEMMA-businessobject.,{C05E56B7-94C2-48D0-B7AE-3EC422A58477}
251,187294,5674,GEMMA Businessobject,NaN,Is opgenomen als businessobject in de GEMMA-ar...,{83A5B969-17C8-4AFA-95F9-01A2EB457F9A}
279,187303,5679,GEMMA Businessobject,Nee,Is opgenomen als businessobject in de GEMMA-ar...,{80B422BD-D727-4D86-A87C-0B3FEA579369}
...,...,...,...,...,...,...
7423,185288,5655,GEMMA uri,https://gemmaonline.staging.wikixl.nl/index.ph...,Uri naar het GEMMA-businessobject.,{82A39422-FA49-49A4-AC5A-1FDBE62118F2}
7424,185289,5656,GEMMA Businessobject,Nee,Is opgenomen als businessobject in de GEMMA-ar...,{19ED8518-8781-4133-9B9A-E163AE76EC93}
7425,185290,5656,GEMMA uri,https://gemmaonline.staging.wikixl.nl/index.ph...,Uri naar het GEMMA-businessobject.,{2B92372B-44D3-4C99-B2FE-1E1D002CC3E3}
7439,185309,5644,GEMMA uri,NaN,Uri naar het GEMMA-businessobject.,{0491FC82-B148-4A3B-B613-CB8E1A5D7D35}
